In [2]:
# Ensure the project root is in the Python path
import sys
sys.path.append('/Users/tl759k/Documents/GitHub/work/cursor-analytics')

# Activate the virtual environment (if not already activated)
# This is typically done outside the notebook, but we ensure the path is correct here
import os
os.system('source /Users/tl759k/Documents/GitHub/work/cursor-analytics/venv/bin/activate')

# Re-import the SnowflakeHook after ensuring the path is set
from utils.snowflake_connection import SnowflakeHook

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import plotly.graph_objects as go
import seaborn as sns
import statsmodels.formula.api as smf
from decimal import Decimal
import plotly.express as px


# Create the Snowflake connection hook once
snowhook = SnowflakeHook()

# Function for queries
def load_query(file_path):
    with open(file_path, 'r') as file:
        query = file.read()
    df = snowhook.query_snowflake(query, method='pandas')

    # format Decimal -> float
    for col in df.columns:
        if df[col].dtype == 'object' and isinstance(df[col].dropna().iloc[0], Decimal):
            df[col] = df[col].astype(float)
    return df


2025-09-10 15:43:41,754 - utils.snowflake_connection - ERROR - Failed to create optimized Spark session: Java gateway process exited before sending its port number
2025-09-10 15:43:41,755 - utils.snowflake_connection - ERROR - Failed to create Spark session: Java gateway process exited before sending its port number


The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

/Users/tl759k/Documents/GitHub/work/cursor-analytics/venv/lib/python3.11/site-packages/pyspark/bin/spark-class: line 96: CMD: bad array subscript
head: illegal line count -- -1


### U.S. Population

In [4]:
df_us_population_raw = pd.read_csv('./data/external/cbsa-est2024-agesex.csv')

df_us_population_raw.columns = df_us_population_raw.columns.str.lower()
# df_us_population_msa = df_us_population[df_us_population['lsad'] == 'Metropolitan Statistical Area'] # 387 MSAs
df_us_population_310 = df_us_population_raw[df_us_population_raw['sumlev'] == 310] # 925 MSAs -- use this version to match with internal MSA list
df_us_population_310_2024 = df_us_population_310[df_us_population_310['year'] == 6] # year 6 is 2024 estimates
df_us_population_310_2024 = df_us_population_310_2024[['sumlev', 'cbsa', 'mdiv', 'name','lsad','year', 'age18plus_tot', 'age1824_tot', 'age2544_tot', 'age4564_tot', 'age65plus_tot']]
df_us_population_310_2024['cbsa'] = df_us_population_310_2024['cbsa'].astype(str)
df_us_population_310_2024.sort_values(by='age18plus_tot', ascending=False).head(10)

,sumlev,cbsa,mdiv,name,lsad,year,age18plus_tot,age1824_tot,age2544_tot,age4564_tot,age65plus_tot
1607,310,35620,NaN,"New York-Newark-Jersey City, NY-NJ",Metropolitan Statistical Area,6,15805908,1676878,5549321,5009533,3570176
1349,310,31080,NaN,"Los Angeles-Long Beach-Anaheim, CA",Metropolitan Statistical Area,6,10339763,1156334,3778057,3279919,2125453
437,310,16980,NaN,"Chicago-Naperville-Elgin, IL-IN",Metropolitan Statistical Area,6,7406901,844918,2635081,2330857,1596045
563,310,19100,NaN,"Dallas-Fort Worth-Arlington, TX",Metropolitan Statistical Area,6,6305189,778987,2482925,1987100,1056177
1025,310,26420,NaN,"Houston-Pasadena-The Woodlands, TX",Metropolitan Statistical Area,6,5816478,729057,2261545,1818224,1007652
1445,310,33100,NaN,"Miami-Fort Lauderdale-West Palm Beach, FL",Metropolitan Statistical Area,6,5186071,512174,1707555,1696926,1269416
1751,310,37980,NaN,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Metropolitan Statistical Area,6,5004008,564909,1740211,1551123,1147765
2405,310,47900,NaN,"Washington-Arlington-Alexandria, DC-VA-MD-WV",Metropolitan Statistical Area,6,5003124,554936,1859627,1616358,972203
113,310,12060,NaN,"Atlanta-Sandy Springs-Roswell, GA",Metropolitan Statistical Area,6,4943007,590814,1808967,1630895,912331
269,310,14460,NaN,"Boston-Cambridge-Newton, MA-NH",Metropolitan Statistical Area,6,4072057,496907,1428850,1246434,899866


#### Zip code to CBSA mapping

Uploaded to snowflake: https://doordash-dev.cloud.databricks.com/editor/notebooks/3377818192540517?o=1953101234786001#command/5114396823128927

table: PRODDB.STATIC.TBL_ZIPCODE_CBSA_MAPPING

In [ ]:
df_zipcode_cbsa = pd.read_excel('data/external/ZIP_CBSA_062025.xlsx', dtype={'ZIP': str, 'CBSA': str})  # force ZIP column to be string
df_zipcode_cbsa['ZIP'] = df_zipcode_cbsa['ZIP'].astype(str)
df_zipcode_cbsa['STATE_NAME'] = df_zipcode_cbsa['USPS_ZIP_PREF_STATE'].astype(str)
# df_zipcode_cbsa.head()

df_zipcode_cbsa = df_zipcode_cbsa[['ZIP','CBSA','STATE_NAME']]
df_zipcode_cbsa.to_csv('data/external/df_zipcode_cbsa.csv', index=False)

In [130]:
df_zipcode_cbsa[df_zipcode_cbsa['CBSA'] != '99999']

,ZIP,CBSA,STATE_NAME
0,00501,35620,NY
2,00601,29720,PR
3,00602,10380,PR
4,00603,10380,PR
5,00604,10380,PR
...,...,...,...
47606,99801,27940,AK
47607,99811,27940,AK
47609,99824,27940,AK
47621,99901,28540,AK


#### Unemployment Rate by MSA

In [131]:
# df_ur_by_msa_raw = pd.read_excel("data/external/msa_unemployment_rate.xlsx")
df_ur_by_msa_raw = pd.read_excel("data/external/msa_unemployment_rate.xlsx", header=2, skipfooter=5, dtype={'year': int, 'month': int})

In [ ]:
df_ur_by_msa = df_ur_by_msa_raw.copy()
df_ur_by_msa.columns = df_ur_by_msa.columns.str.replace(' ', '_')
df_ur_by_msa.columns = df_ur_by_msa.columns.str.lower()
df_ur_by_msa.dropna(inplace=True)
df_ur_by_msa['ur'] = df_ur_by_msa['unemployment_rate']
df_ur_by_msa['msa'] = df_ur_by_msa['area'].str.replace('\\s*MSA$', '', regex=True)
# df_ur_by_msa['date'] = df_ur_by_msa['year'].round().astype(int).astype(str) + '-' + df_ur_by_msa['month'].round().astype(int).astype(str)
df_ur_by_msa['date'] = df_ur_by_msa.apply(
    lambda x: f"{int(x['year']):04d}-{int(x['month']):02d}-01", axis=1
)
df_ur_by_msa = df_ur_by_msa[['date','msa','ur','civilian_labor_force', 'employment', 'unemployment']]
df_ur_by_msa = df_ur_by_msa[df_ur_by_msa['date'] >= '2023-01-01'].reset_index(drop=True)
df_ur_by_msa.rename(columns={'date': 'month'}, inplace=True)

# df_ur_by_msa['state_abbr'] = df_ur_by_msa['msa'].str.split(',').str[1].str.strip()
# df_ur_by_msa = df_ur_by_msa[~df_ur_by_msa['state_abbr'].isin(['PR'])]
df_ur_by_msa

,month,msa,ur,civilian_labor_force,employment,unemployment,state_abbr
0,2023-01-01,"Anniston-Oxford, AL",2.3,49175,48051,1124,AL
1,2023-01-01,"Auburn-Opelika, AL",2.2,92671,90660,2011,AL
2,2023-01-01,"Birmingham, AL",2.1,558178,546516,11662,AL
3,2023-01-01,"Daphne-Fairhope-Foley, AL",1.9,111910,109774,2136,AL
4,2023-01-01,"Decatur, AL",1.8,73279,71924,1355,AL
...,...,...,...,...,...,...,...
12172,2025-07-01,"Racine-Mount Pleasant, WI",3.7,99966,96290,3676,WI
12173,2025-07-01,"Sheboygan, WI",2.7,63470,61767,1703,WI
12174,2025-07-01,"Wausau, WI",2.7,74510,72518,1992,WI
12175,2025-07-01,"Casper, WY",3.3,41498,40135,1363,WY


In [133]:
df_ur_by_msa.to_csv('data/external/df_ur_by_msa.csv', index=False)


#### CPI by MSA

In [134]:
df_cpi_by_state_raw = pd.read_csv('data/external/df_monthly_cpi_by_state.csv')

In [135]:
df_cpi_by_state_raw = df_cpi_by_state_raw[['date','state_name','state_abbr','cpi_value']]
df_cpi_by_state_raw.rename(columns={'date': 'month'}, inplace=True)

In [136]:
df_zipcode_cbsa.columns = df_zipcode_cbsa.columns.str.lower()
df_zipcode_cbsa.rename(columns={'state_name': 'state_abbr'}, inplace=True)

df_cbsa_state = df_zipcode_cbsa[['cbsa','state_abbr']].drop_duplicates()
df_cbsa_state = df_cbsa_state[(df_cbsa_state['cbsa'] != '99999') & (~df_cbsa_state['state_abbr'].isin(['PR']))]
df_cpi_by_msa = df_cpi_by_state_raw.merge(df_cbsa_state, on='state_abbr', how='outer').reset_index(drop=True)
df_cpi_by_msa

,month,state_name,state_abbr,cpi_value,cbsa
0,2023-01-01,Alabama,AL,299.520571,13820
1,2023-01-01,Alabama,AL,299.520571,46220
2,2023-01-01,Alabama,AL,299.520571,10760
3,2023-01-01,Alabama,AL,299.520571,45180
4,2023-01-01,Alabama,AL,299.520571,33860
...,...,...,...,...,...
30809,2025-07-01,Wyoming,WY,324.130395,40540
30810,2025-07-01,Wyoming,WY,324.130395,16220
30811,2025-07-01,Wyoming,WY,324.130395,23940
30812,2025-07-01,Wyoming,WY,324.130395,43260


In [137]:
df_cpi_by_msa.to_csv('data/external/df_monthly_cpi_by_msa.csv', index=False)

#### Minimum Wage by MSA

In [138]:
df_min_wage_by_state = pd.read_csv('data/external/df_monthly_minimum_wage_by_state.csv')
df_min_wage_by_state.columns = df_min_wage_by_state.columns.str.lower()


In [139]:
df_min_wage_by_state = df_min_wage_by_state[~df_min_wage_by_state['state_abbr'].isna()]
df_min_wage_by_state = df_min_wage_by_state[['state_name','state_abbr', 'date','minimum_wage']]

df_min_wage_by_state.rename(columns={'date': 'month', 'minimum_wage': 'min_wage'}, inplace=True)

df_min_wage_by_msa = df_min_wage_by_state.merge(df_cbsa_state, on='state_abbr', how='outer')

In [140]:
df_min_wage_by_msa.to_csv('data/external/df_monthly_min_wage_by_msa.csv', index=False)

#### Compile Economics File

df_min_wage_by_msa, df_cpi_by_msa, df_ur_by_msa

In [141]:
df_min_wage_by_msa

,state_name,state_abbr,month,min_wage,cbsa
0,Alaska,AK,2023-01-01,10.85,11260
1,Alaska,AK,2023-01-01,10.85,21820
2,Alaska,AK,2023-01-01,10.85,27940
3,Alaska,AK,2023-01-01,10.85,28540
4,Alaska,AK,2023-02-01,10.85,11260
...,...,...,...,...,...
30809,Tennessee,TN,2025-07-01,7.25,20540
30810,Tennessee,TN,2025-07-01,7.25,46460
30811,Tennessee,TN,2025-07-01,7.25,37540
30812,Tennessee,TN,2025-07-01,7.25,32280


In [144]:
df_cpi_by_msa

,month,state_name,state_abbr,cpi_value,cbsa
0,2023-01-01,Alabama,AL,299.520571,13820
1,2023-01-01,Alabama,AL,299.520571,46220
2,2023-01-01,Alabama,AL,299.520571,10760
3,2023-01-01,Alabama,AL,299.520571,45180
4,2023-01-01,Alabama,AL,299.520571,33860
...,...,...,...,...,...
30809,2025-07-01,Wyoming,WY,324.130395,40540
30810,2025-07-01,Wyoming,WY,324.130395,16220
30811,2025-07-01,Wyoming,WY,324.130395,23940
30812,2025-07-01,Wyoming,WY,324.130395,43260


In [145]:
df_cpi_by_msa

,month,state_name,state_abbr,cpi_value,cbsa
0,2023-01-01,Alabama,AL,299.520571,13820
1,2023-01-01,Alabama,AL,299.520571,46220
2,2023-01-01,Alabama,AL,299.520571,10760
3,2023-01-01,Alabama,AL,299.520571,45180
4,2023-01-01,Alabama,AL,299.520571,33860
...,...,...,...,...,...
30809,2025-07-01,Wyoming,WY,324.130395,40540
30810,2025-07-01,Wyoming,WY,324.130395,16220
30811,2025-07-01,Wyoming,WY,324.130395,23940
30812,2025-07-01,Wyoming,WY,324.130395,43260


In [142]:
df_econ_by_msa = df_ur_by_msa.merge(df_cpi_by_msa, on=['state_abbr','month'], how='left').reset_index(drop=True)
df_econ_by_msa = df_econ_by_msa.merge(df_min_wage_by_msa, on=['state_abbr','month','cbsa','state_name'], how='left').reset_index(drop=True)
df_econ_by_msa

,month,msa,ur,civilian_labor_force,employment,unemployment,state_abbr,state_name,cpi_value,cbsa,min_wage
0,2023-01-01,"Anniston-Oxford, AL",2.3,49175,48051,1124,AL,Alabama,299.520571,13820,7.25
1,2023-01-01,"Anniston-Oxford, AL",2.3,49175,48051,1124,AL,Alabama,299.520571,46220,7.25
2,2023-01-01,"Anniston-Oxford, AL",2.3,49175,48051,1124,AL,Alabama,299.520571,10760,7.25
3,2023-01-01,"Anniston-Oxford, AL",2.3,49175,48051,1124,AL,Alabama,299.520571,45180,7.25
4,2023-01-01,"Anniston-Oxford, AL",2.3,49175,48051,1124,AL,Alabama,299.520571,33860,7.25
...,...,...,...,...,...,...,...,...,...,...,...
305407,2025-07-01,"Cheyenne, WY",3.3,48575,46969,1606,WY,Wyoming,324.130395,40540,5.15
305408,2025-07-01,"Cheyenne, WY",3.3,48575,46969,1606,WY,Wyoming,324.130395,16220,5.15
305409,2025-07-01,"Cheyenne, WY",3.3,48575,46969,1606,WY,Wyoming,324.130395,23940,5.15
305410,2025-07-01,"Cheyenne, WY",3.3,48575,46969,1606,WY,Wyoming,324.130395,43260,5.15


In [143]:
df_econ_by_msa.to_csv('data/external/df_econ_by_msa.csv', index=False)